In [ ]:
# Импорт библиотек
import cv2 # для работы с компьютерным зрением
import os # для работы с Windows
import matplotlib.pyplot as plt  # библиотеки вывода изображений
import tensorflow as tf # для работы с нейросетью
import keras # для работы с нейросетью

# Визуализируем real and fake faces в dataset

real = "D:/Python_Projects/Computer_Vision/DataSet/train/real/"
fake = "D:/Python_Projects/Computer_Vision/DataSet/train/fake/"
real_path = os.listdir(real)
fake_path = os.listdir(fake)

def load_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (100, 100))
    return image[..., ::-1]

figure = plt.figure(figsize=(8, 8))

for i in range(8):
    plt.subplot(4, 4, i + 1)
    plt.imshow(load_img(real + real_path[i]), cmap='gray')
    plt.suptitle("Real faces", fontsize=18)
    plt.axis('off')
plt.show()

def load_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (100, 100))
    return image[..., ::-1]

figure = plt.figure(figsize=(8, 8))

for i in range(8):
    plt.subplot(4, 4, i + 1)
    plt.imshow(load_img(fake + fake_path[i]), cmap='gray')
    plt.suptitle("Fake faces", fontsize=18)
    plt.axis('off')
plt.show()

# Загружаем данные dataset и смотрим, что они загрузились

real = os.path.join('D:/Python_Projects/Computer_Vision/DataSet/train/real/')
fake = os.path.join('D:/Python_Projects/Computer_Vision/DataSet/train/fake/')

real = os.listdir(real)
print(real[:10])
fake = os.listdir(fake)
print(fake[:10])

print('Общее количество real:', len(real))
print('Общее количество fake:', len(fake))

In [ ]:
import torch
device = torch.device 
if torch.cuda.is_available:
    device = torch.device
    print("Running on the GPU")
else:
    device = torch.device
    print("Running on the CPU")

In [ ]:
# Импорт библиотек
import matplotlib.pyplot as plt  # библиотеки вывода изображений
from keras.models import Sequential  # общий класс, последовательная модель
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, BatchNormalization, Dropout  #
# для работы с полносвязными слоями, для конвертации входящих данных в меньшую размерность, для выбора максимума из
# пикселей, покрытых фильтром (для решении проблемы чувствительности), для сверточных слоев, для стабилизации работы
#нейросети, для регуляризации
from keras.preprocessing.image import ImageDataGenerator #для приведения диапазона пикселей к воспринимаемому
from tensorflow.keras.applications import VGG19  # импорт модели VGG19

# Загружаем VGG19 и добавляем слои

vgg19_network = VGG19(input_shape=(256, 256, 3), include_top=False, weights='imagenet')

vgg19new = Sequential()
vgg19new.add(vgg19_network)

for layer in vgg19new.layers:  # Зафиксируем веса на каждом слое, чтобы они не обновлялись
    layer.trainable = False

vgg19new.add(Flatten())
vgg19new.add(Dense(128, activation='relu',kernel_initializer='he_normal'))
vgg19new.add(Dense(32, activation='relu',kernel_initializer='he_normal'))
vgg19new.add(Dense(8, activation='relu',kernel_initializer='he_normal'))
vgg19new.add(Dense(1, activation='sigmoid'))

# Компилируем полученную модель

vgg19new.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
vgg19new.summary()

# Задаем данные для обучения

data_generator = ImageDataGenerator(rescale=1 / 255)  # чтобы привести диапазон пикселей к диапазону [0, 1],

train_data = data_generator.flow_from_directory('D:/Python_Projects/Computer_Vision/DataSet/train',
                                                target_size=(256, 256), batch_size=32, class_mode='binary')

# Задаем данные для валидации результатов и для проверки

val = data_generator.flow_from_directory('D:/Python_Projects/Computer_Vision/DataSet/valid',
                                         target_size=(256, 256), batch_size=32, class_mode='binary')

test = data_generator.flow_from_directory('D:/Python_Projects/Computer_Vision/DataSet/test',
                                          target_size=(256, 256), batch_size=32, class_mode='binary')

# Обучаем модель

train = vgg19new.fit(train_data, epochs=100, steps_per_epoch=5, validation_data=val)

# Сохраняем модель

vgg19new.save("compvision")

# Визуализация функции точности и функции потерь

epochs = 100
train_loss = train.history['loss']
val_loss = train.history['val_loss']
train_acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
r = range(epochs)

plt.figure(1, figsize=(8, 5))
plt.plot(r, train_loss)
plt.plot(r, val_loss)
plt.xlabel('Количество эпох')
plt.ylabel('Значение loss')
plt.title('График Train loss VS Val loss')
plt.grid(True)
plt.legend(['train', 'val'])
plt.style.use(['seaborn-bright'])
plt.show()

plt.figure(2, figsize=(8, 5))
plt.plot(r, train_acc)
plt.plot(r, val_acc)
plt.xlabel('Количество эпох')
plt.ylabel('Значение accuracy')
plt.title('График Train accuracy VS Val accuracy')
plt.grid(True)
plt.legend(['train', 'val'])
plt.style.use(['seaborn-bright'])
plt.show()

# Оценка точности модели

loss, accuracy = vgg19new.evaluate(test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)